In [1]:
txt = '''I think that the planet of the little prince is Asteroid B-612.
In 1909, one astronomer discovers this asteroid. The astronomer is from Turkey.

He speaks about his asteroid at a big conference. But nobody believes him because he has Turkish clothes. It is crazy but people do these things.

But there is something good for Asteroid B-612. A Turkish leader says that people in Turkey must have clothes like Europeans. The astronomer speaks about his asteroid again in 1920. He has very nice clothes. And now everybody believes him.

I speak about Asteroid B-612 and I speak about its number because people like numbers. When you tell people that you have a new friend, they never ask you questions about important things. They never ask you, “Is his voice nice? What games does he play? Does he have butterflies at home?”

They ask, “How old is he? How many brothers does he have? How big is he? How much money does his father have?” If they know these numbers, they think that they know this person.

If you say to the people, “I know a small red house. It has beautiful flowers in the windows. A lot of birds fly around the house,” the people can’t imagine the house. You must say to the people, “I know a big house. It is very expensive. You can buy this house for one hundred thousand dollars.” Then the people say, “It is a nice house.”

If you say to the people, “The little prince exists because he laughs and he wants a sheep,” this information is not enough for the adults. They don’t believe you. They think that you live in a dream. But if you tell them, “The planet of the little prince is Asteroid B-612,” they believe you. This is how the people think.

If you understand life, you don’t need numbers. You need to hear a nice story. You like to hear, “The little prince lives on a small planet. The planet is very small. The little prince is alone. He wants to have a friend.” If you understand life, this is what you want to hear.'''

In [2]:
## Make nested word_list
import re
from graphbrain.parsers import *

parser = create_parser(lang='en')

def split_words(text=''):
    words = re.split('(\W)',text)
    words = list(filter(lambda w: (w != ''), words))
    last = ''
    result = []
    for word in words:
        if last == "'":
            match = re.match('s|re|m|ve|d|t|ll|clock', word, re.I)
            if match:
                word = result.pop() + word
                if match[0] == 't' and re.match('n',result[-1][-1], re.I):
                    last = list(result.pop())
                    word = last.pop() + word
                    result.append(''.join(last)) 
        result.append(word)
        last = word
    return result

def make_paragraphs_array(text=''):
    paragraphs_list = text.splitlines()
    paragraphs = []
    for p in paragraphs_list:
        parses = parser.parse(p)['parses']
        sentences = []
        for sentence in parses:
            sentences.append(sentence['text'])
        sentences = list(map(lambda s: split_words(s), sentences),)
        paragraphs.append(sentences)
    return(paragraphs)

In [16]:
import pandas as pd
common_words = pd.read_excel('common_words.ods')
common_words = common_words[common_words['lemma'] != "It"]

In [29]:

common = common_words[['lemma','word']].dropna()
common.set_index('word')

,lemma
word,
the,the
ze,the
is,be
was,be
's,be
...,...
wheat,wheat
wheats,wheat
predators,predator


In [30]:
words = common['word'].to_list()
lemmas = common['lemma'].to_list()
dictionary = {}
for i, w in enumerate(words):
    dictionary[w] = lemmas[i]
dictionary



{'the': 'the',
 'ze': 'the',
 'is': 'be',
 'was': 'be',
 "'s": 'be',
 'be': 'be',
 'are': 'be',
 'were': 'be',
 "'re": 'be',
 'been': 'be',
 "'m": 'be',
 'being': 'being',
 'am': 'am',
 's': 'be',
 'and': 'and',
 'a': 'a',
 'an': 'a',
 'of': 'of',
 'to': 'to',
 'na': 'to',
 'ta': 'to',
 'in': 'in',
 'i': 'I',
 'you': 'you',
 'ya': 'you',
 'youse': 'you',
 'yous': 'you',
 'it': 'it',
 'have': 'have',
 'had': 'have',
 'has': 'have',
 "'ve": 'have',
 "'d": 'have',
 'having': 'have',
 'hath': 'have',
 'haveing': 'have',
 'that': 'that',
 'for': 'for',
 'do': 'do',
 'did': 'do',
 'does': 'do',
 'doing': 'do',
 'done': 'do',
 'doin': 'do',
 'doth': 'do',
 'don': 'do',
 'he': 'he',
 'with': 'with',
 'on': 'on',
 'this': 'this',
 "n't": "n't",
 'we': 'we',
 'not': 'not',
 'but': 'but',
 'they': 'they',
 'said': 'say',
 'say': 'say',
 'says': 'say',
 'saying': 'say',
 'at': 'at',
 'what': 'what',
 'his': 'his',
 'from': 'from',
 'go': 'go',
 'going': 'go',
 'went': 'go',
 'gon': 'go',
 'gone': 

In [31]:
with open('lemmas.json', 'w') as f:
    json.dump(dictionary, f, ensure_ascii = False)

In [ ]:
words = common_words['word']
ocurrences = []
for word in words:
    if word in ocurrences:
        lemmas = common_words[common_words['word'] == word]['lemma'].unique()
        if len(lemmas) > 1:
            print(word)
    ocurrences.append(word)


In [4]:
### construct map for words ocurrenses for a given text

def make_words_map(nestet_text = []):
    text_data = {}
    p_index = 0
    for paragraph in nestet_text:
        s_index = 0
        for sentence in paragraph:
            for word in  sentence:
                if re.match('\w+', word):
                    lemma = common_words[common_words['word'] == word.lower()]['lemma'].unique()
                    if len(lemma) > 1:
                        if 'ambiguities' not in text_data:
                            text_data['ambiguities'] = {} 
                            text_data['ambiguities']['paragraphs'] = []
                            text_data['ambiguities']['sentences'] = []
                            text_data['ambiguities']['word'] = []
                        text_data['ambiguities']['paragraphs'].append(p_index)
                        text_data['ambiguities']['sentences'].append(s_index)
                        text_data['ambiguities']['word'].append(word)
                    elif len(lemma) > 0:
                        lemma = lemma[0]
                        if lemma not in text_data:
                            text_data[lemma] = {}
                            text_data[lemma]['paragraph'] = []
                            text_data[lemma]['sentence'] = []
                        text_data[lemma]['paragraph'].append(p_index)
                        text_data[lemma]['sentence'].append(s_index)
            s_index += 1
        p_index += 1
    return text_data

In [11]:
import json

with open('the_little_prince.json', 'r') as f:
    text_file = json.loads(f.read())


In [6]:
chapter_4 = {}
chapter_4['title'] = 'asteroid'
nested = make_paragraphs_array(txt.strip())
chapter_4['text'] = txt
chapter_4['words_map'] = make_words_map(nested)
chapter_4['nested'] = nested
text_file['chapters'].append(chapter_4)


In [15]:
import json

with open ('the_little_prince.json', 'w') as f:
    json.dump(text_file, f, ensure_ascii = False)